# Домашнее задание

И пусть твой бот заговорит!

## Цель:

В этом ДЗ вы создадите телеграм-бота.

### Описание/Пошаговая инструкция выполнения домашнего задания:

1. Возьмите ОДНУ задачу из бенчмарка [Russian SuperGLUE](https://russiansuperglue.com/tasks/)

2. Зафайнтьюньте для него модель с сайта huggingface (например, модель RuBERT).

> Замечание:
> - не берите задачу RWSD, на ней модели склонны вырождаться в константу;
> - не берите очень большие модели, иначе их будет сложно встроить в бота.

3. Сделайте сабмит на [Leaderboard](https://russiansuperglue.com/leaderboard/2) для оценки качества (в ноутбуке прикрепите скрин сабмита с результатами).

4. Создайте телеграм-бота, в котором будет поднята эта модель. Бот должен вначале писать вводное сообщение с описанием задачи, обрабатывать запрос пользователя и выводить ответ для заданного пользователем примера.

> Замечание: пользователь должен писать боту запрос с примером из соотвествующего задания RussianSuperGLUE, под которого зафайнтьюнена модель. Возможны разные варианты реализации: пользователь пишет пример со всеми полями в одном сообщении или в нескольких. Конкретная реализация за вами.

> Замечание: для проверки можете отправить код с телеграм-ботом и приложить скрины с примерами его работы (в случае если нет возможности поддерживать его в поднятом состоянии).

In [3]:
import random
import time
from tqdm import tqdm
import gc

import numpy as np
import pandas as pd

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW

from transformers import BertTokenizer, BertForSequenceClassification, BertConfig  
from transformers import get_linear_schedule_with_warmup

import requests
import telebot
from telebot import types

import warnings
warnings.filterwarnings("ignore")

# 1. Выбор задачи и модели

В качестве задачи выберем "Yes/no Question Answering Dataset for the Russian", в которой модели на вход подаются контекст и вопрос, а от модели ожидается ответ Да/Нет. Соответственно, будем решать задачу бинарной классификации, а в качестве модели возьмем ruBert-base из репозитория ai-forever

In [6]:
model_ckpt = "ai-forever/ruBert-base"

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
tokenizer = BertTokenizer.from_pretrained(model_ckpt, do_lower_case=True)
tokenizer

BertTokenizer(name_or_path='ai-forever/ruBert-base', vocab_size=120138, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [9]:
model = BertForSequenceClassification.from_pretrained(
    model_ckpt, 
    num_labels = 2, 
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
# Tell pytorch to run this model on the GPU.
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [10]:
learning_rate = 2e-5
optimizer = AdamW(model.parameters(), lr=learning_rate) 

In [11]:
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 2e-05
    maximize: False
    weight_decay: 0.01
)

In [12]:
def load_data(tokenizer, questions, passages, max_length):
    """Encode the question/passage pairs into features than can be fed to the model."""
    input_ids = []
    attention_masks = []

    for question, passage in zip(questions, passages):
        encoded_data = tokenizer.encode_plus(question, passage, max_length=max_length, pad_to_max_length=True, truncation_strategy="longest_first")
        encoded_pair = encoded_data["input_ids"]
        attention_mask = encoded_data["attention_mask"]

        input_ids.append(encoded_pair)
        attention_masks.append(attention_mask)
    return np.array(input_ids), np.array(attention_masks)

### Загружаем данные для обучения и валидации из датасета [Russian SuperGLUE](https://russiansuperglue.com/tasks/)

In [14]:
train_data_df = pd.read_json('./data/DaNetQA/train.jsonl', lines=True)
train_data_df.head()

,question,passage,label,idx
0,Вднх - это выставочный центр?,«Вы́ставочный центр» — станция Московского мон...,True,0
1,Вднх - это выставочный центр?,"Вы́ставка достиже́ний наро́дного хозя́йства ,...",True,1
2,Был ли джиган в black star?,Вместе с этим треком они выступили на церемони...,True,2
3,Xiaomi конкурент apple?,"Xiaomi — китайская компания, основанная в 2010...",True,3
4,Был ли автомат калашникова в вов?,Отметив некоторые недостатки и в целом удачную...,False,4


In [15]:
dev_data_df = pd.read_json('./data/DaNetQA/val.jsonl', lines=True)
dev_data_df.head()

,question,passage,label,idx
0,Есть ли вода на марсе?,Гидросфера Марса — это совокупность водных зап...,True,0
1,Состоит ли англия в евросоюзе?,В полночь с 31 января на 1 февраля 2020 года п...,False,1
2,Действительно ли в ссср не было адвокатов?,Семён Львович Ария — советский и российский ю...,False,2
3,Была ли чума в оране?,"Чума — это и абсурд, что осмысливается как фор...",True,3
4,Был ли кетчуп в читосе?,Текущий каталог продукции размещен на сайте пр...,True,4


In [16]:
passages_train = train_data_df.passage.values
questions_train = train_data_df.question.values
answers_train = train_data_df.label.values.astype(int)
print(answers_train)

passages_dev = dev_data_df.passage.values
questions_dev = dev_data_df.question.values
answers_dev = dev_data_df.label.values.astype(int)

# Encoding data
max_seq_length = 256
input_ids_train, attention_masks_train = load_data(tokenizer, questions_train, passages_train, max_seq_length)
input_ids_dev, attention_masks_dev = load_data(tokenizer, questions_dev, passages_dev, max_seq_length)

train_features = (input_ids_train, attention_masks_train, answers_train)
dev_features = (input_ids_dev, attention_masks_dev, answers_dev)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[1 1 1 ... 1 0 1]


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [17]:
# Building Dataloaders
batch_size = 32

train_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in train_features]
dev_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in dev_features]

train_dataset = TensorDataset(*train_features_tensors)
dev_dataset = TensorDataset(*dev_features_tensors)

train_sampler = RandomSampler(train_dataset)
dev_sampler = SequentialSampler(dev_dataset)

train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
dev_dataloader = DataLoader(dev_dataset, sampler=dev_sampler, batch_size=batch_size)

Предобработка данных завершена, переходим к обучению модели

## 2. Fine tuning ruBert-base

In [20]:
gc.collect()
torch.cuda.empty_cache()

In [21]:
# Number of EPOCHS
EPOCHS = 4
total_steps = len(train_dataloader) * EPOCHS
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [22]:
def accuracy(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [23]:
# seting initial best loss to infinite
best_eval_loss = float('inf')

#for _ in tqdm(range(epochs), desc="Epoch"):
for epoch in range(EPOCHS):

  # Training
  model.train()

  # Reset the total loss and accuracy for this epoch.
  total_train_loss = 0
  total_train_acc = 0
  
  # Measure how long the training epoch takes.
  start = time.time()

  for step, batch in enumerate(train_dataloader):
      # Unpack this training batch from our dataloader.
      input_ids = batch[0].to(device)
      attention_masks = batch[1].to(device)
      labels = batch[2].to(device)  

      #clear any previously calculated gradients before performing a backward pass
      optimizer.zero_grad()

      #result = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)
      loss, prediction = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels).values()
      #loss = result.loss
      #logits = result.logits
      acc = accuracy(prediction, labels)

      # Accumulate the training loss and accuracy over all of the batches so that we can
      # calculate the average loss at the end
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

      # Perform a backward pass to calculate the gradients.
      loss.backward()

      # Clip the norm of the gradients to 1.0.
      # This is to help prevent the "exploding gradients" problem.
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      # Update parameters and take a step using the computed gradient.
      optimizer.step()

      # Update the learning rate.
      scheduler.step()

  # Calculate the average accuracy and loss over all of the batches.
  train_acc  = total_train_acc/len(train_dataloader)
  train_loss = total_train_loss/len(train_dataloader)

  # Put the model in evaluation mode
  model.eval()

  total_val_acc  = 0
  total_val_loss = 0

  with torch.no_grad():
    for batch in dev_dataloader:

      #clear any previously calculated gradients before performing a backward pass
      optimizer.zero_grad()

      # Unpack this validation batch from our dataloader.
      input_ids = batch[0].to(device)
      attention_masks = batch[1].to(device)
      labels = batch[2].to(device) 

      #Get the loss and prediction
      loss, prediction = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels).values()

      # Calculate the accuracy for this batch
      acc = accuracy(prediction, labels)

      # Accumulate the validation loss and Accuracy
      total_val_loss += loss.item()
      total_val_acc  += acc.item()

  # Calculate the average accuracy and loss over all of the batches.
  val_acc  = total_val_acc/len(dev_dataloader)
  val_loss = total_val_loss/len(dev_dataloader)

  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)

  print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

  #save the best model
  if val_loss < best_eval_loss:
      best_eval_loss = val_loss
      torch.save(model.state_dict(), 'ruBert_saved_weights.pt')     

Epoch 1: train_loss: 0.6069 train_acc: 0.6840 | val_loss: 0.7503 val_acc: 0.5623
00:00:10.22
Epoch 2: train_loss: 0.5181 train_acc: 0.7496 | val_loss: 0.6863 val_acc: 0.6068
00:00:09.69
Epoch 3: train_loss: 0.4223 train_acc: 0.8079 | val_loss: 0.8316 val_acc: 0.5816
00:00:09.51
Epoch 4: train_loss: 0.3361 train_acc: 0.8676 | val_loss: 0.8922 val_acc: 0.5816
00:00:09.24


In [24]:
gc.collect()
torch.cuda.empty_cache()

In [25]:
#loading the best saved model
path = 'ruBert_saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [26]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

### Проверим на тестовых данных, что предсказывает модель

In [28]:
# Load Test data
test_data_df = pd.read_json('./data/DaNetQA/test.jsonl', lines=True)
test_data_df.head()

,question,passage,idx
0,Полезна ли ртуть с градусника?,"Отравления ртутью — расстройства здоровья, св...",0
1,Являются ли сапрофаги хищниками?,Фауна лесных почв — совокупность видов животны...,1
2,Водятся ли в индии крокодилы?,"Болотный крокодил, или магер — пресмыкающееся...",2
3,Есть ли в батате крахмал?,"Клубневидно вздутые корни весят до 15 кг, сод...",3
4,Был ли человек в железной маске?,Остров Сент-Маргерит — крупнейший из Лерински...,4


In [29]:
passage_list = test_data_df['passage'].to_list()
question_list = test_data_df['question'].to_list()

In [196]:
# Predict the answer
def predict(question, passage):
  sequence = tokenizer.encode_plus(question, passage, return_tensors="pt")['input_ids'].to(device)
  logits = model(sequence)[0]
  probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
  proba_yes = round(probabilities[1], 3)
  proba_no = round(probabilities[0], 3)
  print(f"Question: {question}, True: {proba_yes}, False: {proba_no}")
  return f"True: {proba_yes}, False: {proba_no}"  

In [31]:
sample_ids = random.sample(range(1, len(question_list)), 10)

#for i in range(1,11):
for i in sample_ids:
    question = question_list[i]
    passage = passage_list[i]
    predict(question,passage)

Question: Входит суффикс л основу слова?, True: 0.605, False: 0.395
Question: Помогает ли алкоголь от рака?, True: 0.561, False: 0.439
Question: Запрещено ли в китае есть палочками?, True: 0.646, False: 0.354
Question: Были ли в Народном комиссариате юстиции юристы до 1918 года?, True: 0.764, False: 0.236
Question: Состоит ли россия в нато?, True: 0.348, False: 0.652
Question: Входит ли хорватия в шенгенскую зону?, True: 0.139, False: 0.861
Question: Есть ли в ушах кости?, True: 0.826, False: 0.174
Question: Есть ли у китая ядерное оружие?, True: 0.756, False: 0.244
Question: Есть ли в дизельном двигателе свечи?, True: 0.773, False: 0.227
Question: Была ли конституция в царской россии?, True: 0.832, False: 0.168


## 3. Submit on [Leaderboard](https://russiansuperglue.com/leaderboard/2)

Напишем функцию получения метки для подготовки файла с результатами и последующей загрузки на Leaderboard

In [34]:
# get the prediction's label for submitting 
def get_label(question, passage):
    sequence = tokenizer.encode_plus(question, passage, return_tensors="pt")['input_ids'].to(device)
    logits = model(sequence)[0]
    probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
    return 'true' if np.argmax(probabilities) else 'false' 

In [35]:
test_data_df.head()

,question,passage,idx
0,Полезна ли ртуть с градусника?,"Отравления ртутью — расстройства здоровья, св...",0
1,Являются ли сапрофаги хищниками?,Фауна лесных почв — совокупность видов животны...,1
2,Водятся ли в индии крокодилы?,"Болотный крокодил, или магер — пресмыкающееся...",2
3,Есть ли в батате крахмал?,"Клубневидно вздутые корни весят до 15 кг, сод...",3
4,Был ли человек в железной маске?,Остров Сент-Маргерит — крупнейший из Лерински...,4


In [36]:
# addinп a column with target label
test_data_df['label'] = ''
test_data_df.head()

,question,passage,idx,label
0,Полезна ли ртуть с градусника?,"Отравления ртутью — расстройства здоровья, св...",0,
1,Являются ли сапрофаги хищниками?,Фауна лесных почв — совокупность видов животны...,1,
2,Водятся ли в индии крокодилы?,"Болотный крокодил, или магер — пресмыкающееся...",2,
3,Есть ли в батате крахмал?,"Клубневидно вздутые корни весят до 15 кг, сод...",3,
4,Был ли человек в железной маске?,Остров Сент-Маргерит — крупнейший из Лерински...,4,


In [37]:
# Predicting target labels for submission
for row in test_data_df.itertuples():
    #print(row.idx, row.question, get_label(row.question, row.passage))  ## debugging
    test_data_df['label'][row.idx] = get_label(row.question, row.passage)

In [38]:
test_data_df.tail()

,question,passage,idx,label
800,Была ли у джихангира наложница?,Хюрре́м Хасеки́-султа́н ; настоящее имя неизве...,800,true
801,Правда ли у животных нет групп крови?,Гру́ппа кро́ви — описание индивидуальных антиг...,801,true
802,Все ли погибли на титанике?,"Элизабет Глэдис Миллвина Дин — англичанка, ко...",802,true
803,Есть ли в хлебе белок?,Эта статья о группе белков. О пищевом продукте...,803,true
804,Правда ли астану переименовали?,Нур-Султа́н — столица Республики Казахстан с ...,804,true


In [39]:
submission_df = test_data_df[['idx','label']]

In [40]:
filepath = './submission/DaNetQA.jsonl'
submission_df.to_json(filepath, orient='records', lines=True)
# submission_df.to_json('./leaderboard/DaNetQA.jsonl.gz', orient='records', lines=True, compression='zip')

![](./img/rusuperglue-submit.png)

Не очень понимаю, что означает этот TOTAL и почему не указана метрика Accuracy для загруженных результатов... RussianSuperGluk

## 4. Telegram Bot

In [44]:
path2token = './OracleBiBot_token.txt'

In [45]:
with open(path2token) as f:
    TOKEN = f.read().strip()

In [46]:
def make_bot_url(path):
    with open(path) as f:
        TOKEN = f.read().strip()
    bot_url = f'https://api.telegram.org/bot{TOKEN}'
    return bot_url

In [47]:
def send_message(text: str, channel_id: int, reply_to: int=None, reply_markup: dict=None):
    """text: string of any length
       channel_id: int - chat id
       reply_to: int message to reply id
       reply_markup: dict {inline_keyboard: [[{ text: 'Stop', callback_data: "string" }]],}
       return: list of response objects. res[0].status_code, res[0].json()
    """
    
    method = make_bot_url(path2token) + "/sendMessage"

    res = []
    if len(text) > 4096:
        for x in range(0, len(text), 4096):
            text_ = text[x:x + 4096]
            data = {
                "chat_id": channel_id,
                "text": text_
            }
            
            if reply_to:
                data['reply_to_message_id'] = reply_to

            r = requests.post(method, data=data)
            res.append(r)
    else:
        data = {
            "chat_id": channel_id,
            "text": text
        }
        
        if reply_markup:
            assert type(reply_markup) == dict
            data['reply_markup'] = json.dumps(reply_markup)
        
        if reply_to:
            data['reply_to_message_id'] = reply_to

        r = requests.post(method, data=data)
        res.append(r)

    return res

In [48]:
text = 'I am binary oracle bot'
my_chat_id = 406137873

In [49]:
r = send_message(text=text, channel_id=my_chat_id)
r

[<Response [200]>]

In [50]:
url = f'https://api.telegram.org/bot{TOKEN}' + "/getMe"

In [51]:
resp = requests.get(url)

In [52]:
resp.json()

{'ok': True,
 'result': {'id': 7662060530,
  'is_bot': True,
  'first_name': 'OracleBI',
  'username': 'OracleBiBot',
  'can_join_groups': True,
  'can_read_all_group_messages': False,
  'supports_inline_queries': False,
  'can_connect_to_business': False,
  'has_main_web_app': False}}

In [191]:
welcome_message = 'Вас приветствует бот, обученный давать бинарные ответы\nОпишите контекст, задайте вопрос с новой строки - и вы получите ответ:'

In [209]:
bot = telebot.TeleBot(TOKEN)

In [211]:
@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, welcome_message)

@bot.message_handler(func=lambda x: True, chat_types=["private"])
def start_message(message):
    print(message.text)
    #bot.send_message(message.chat.id, "<b>Any other</b> <i>handler</i>", parse_mode="HTML")
    splits = message.text.split(sep="\n", maxsplit=1)
    print(f"C: {splits[0].strip()}")
    print(f"Q: {splits[1].strip()}")
    answer = predict(splits[1].strip(), splits[0].strip())
    print(answer)
    bot.send_message(message.chat.id, f"<b>Ответ на вопрос: {splits[1].strip()}</b>\n<i>{answer}</i>", parse_mode="HTML")

bot.infinity_polling()

Свечи накаливания  — детали в дизельном двигателе, в предпусковом подогревателе двигателя, в автономном отопителе салона  и в калильном карбюраторном двигателе, служащие для облегчения его холодного пуска. В отличие от свечей зажигания, они не дают искру, а представляют собой обычный электронагревательный элемент. При нормальной работе дизеля горючее в цилиндрах поджигается путём самовоспламенения в момент впрыска его форсунками в предварительно сжатый поршнем воздух, нагретый до 700—900°С; соответственно, обычных искровых свечей зажигания, как в бензиновом двигателе, в его конструкции нет и не требуется. Свечи накаливания являются по сути частью предпускового устройства двигателя и нужны по большому счёту только для облегчения пуска дизеля после долгого простоя  за счёт улучшения условий образования рабочей смеси, особенно если температура воздуха низка. При такой температуре дизельное топливо испаряется неохотно, и не образует необходимой для работы двигателя смеси с воздухом.
Есть л

2025-04-15 23:09:46,609 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-04-15 23:09:46,610 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"


## Bot Screenshots

![](./img/screenshot-disel.png)

![](./img/screenshot-nato.png)

### Bot Test Examples

In [199]:
example_ids = [109, 589]

In [201]:
test_data_df['passage'][example_ids[0]], test_data_df['question'][example_ids[0]]

('НА́ТО, Организа́ция Североатланти́ческого догово́ра, Североатлантический Альянс  — военно-политический блок, объединяющий большинство стран Европы, США и Канаду. Основан 4 апреля 1949 года в США с целью защиты Европы от советского влияния. Тогда государствами — членами НАТО стали 12 стран: США, Канада, Исландия, Великобритания, Франция, Бельгия, Нидерланды, Люксембург, Норвегия, Дания, Италия и Португалия. Это «трансатлантический форум» для проведения странами-союзниками консультаций по любым вопросам, затрагивающим жизненно важные интересы его членов, включая события, способные поставить под угрозу их безопасность. Одной из заявленных целей НАТО является обеспечение сдерживания любой формы агрессии в отношении территории любого государства — члена НАТО или защиты от неё.',
 'Состоит ли россия в нато?')

In [203]:
predict (test_data_df['question'][example_ids[0]], test_data_df['passage'][example_ids[0]])

Question: Состоит ли россия в нато?, True: 0.348, False: 0.652


'True: 0.348, False: 0.652'

In [205]:
test_data_df['passage'][example_ids[1]], test_data_df['question'][example_ids[1]]

('Свечи накаливания  — детали в дизельном двигателе, в предпусковом подогревателе двигателя, в автономном отопителе салона  и в калильном карбюраторном двигателе, служащие для облегчения его холодного пуска. В отличие от свечей зажигания, они не дают искру, а представляют собой обычный электронагревательный элемент. При нормальной работе дизеля горючее в цилиндрах поджигается путём самовоспламенения в момент впрыска его форсунками в предварительно сжатый поршнем воздух, нагретый до 700—900°С; соответственно, обычных искровых свечей зажигания, как в бензиновом двигателе, в его конструкции нет и не требуется. Свечи накаливания являются по сути частью предпускового устройства двигателя и нужны по большому счёту только для облегчения пуска дизеля после долгого простоя  за счёт улучшения условий образования рабочей смеси, особенно если температура воздуха низка. При такой температуре дизельное топливо испаряется неохотно, и не образует необходимой для работы двигателя смеси с воздухом.',
 '

In [207]:
predict(test_data_df['question'][example_ids[1]], test_data_df['passage'][example_ids[1]])

Question: Есть ли в дизельном двигателе свечи?, True: 0.773, False: 0.227


'True: 0.773, False: 0.227'